In [3]:
import cv2
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('/usr/local/python')
from openpose import pyopenpose as op
import pandas as pd
import os
from tqdm import tqdm
import argparse
import os


In [ ]:
OPENPOSE_MODEL_DIR = "/home/soon/github/openpose/models" # openpose/models

In [2]:
df = pd.read_pickle('../data/deepfashion_1.pickle')

In [ ]:
params = dict()
params["model_folder"] = OPENPOSE_MODEL_DIR
params["disable_blending"] = True
params["keypoint_scale"] = 3 #normalise keypoints to [0,1]
params["num_gpu"] = 1
params["num_gpu_start"] = args.gpu
params["number_people_max"] = args.number_people_max


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--folder', type=str)
parser.add_argument('--output_name', type=str)
parser.add_argument('--number_people_max', type=int, default=4)

args = parser.parse_args()

#folder = os.path.join(args.folder, '*')
folder = args.folder
image_files = glob(folder,recursive=False)
image_files.sort()
print(f"Found {len(image_files)} files.")


opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()


column_names = ["image", "pose_score", "keypoints"]
df = pd.DataFrame(columns=column_names)

for image_path in tqdm(image_files):
    datum = op.Datum()
    imageToProcess = cv2.imread(image_path)
    datum.cvInputData = imageToProcess
    opWrapper.emplaceAndPop(op.VectorDatum([datum]))
    scores = datum.poseScores
    try:
        if scores is not None:
            fname = image_path.replace(args.folder,'')
            row = {'image':fname, 
                   'pose_score':datum.poseScores, 
                   'keypoints':datum.poseKeypoints}
            df = df.append(row, ignore_index=True)
    except Exception as e:
        print(e, image_path)

df.to_pickle(args.output_name)
print(f"Save into {args.output_name}")

